# String
<hr>

### 344 Reverse String

使用双指针法来进行reverse。要求$O(1)$.首位交换，再往中间移动再换。



In [5]:
from typing import List
class Solution:
    def reverseString(self, s: List[str]) -> None:
        """
        Do not return anything, modify s in-place instead.
        """
        # define the length of the string
        n = len(s)

        # define two pointers
        left, right = 0, n-1

        # loop until the two pointers meet
        while left < right:
            # swap the two elements
            s[left], s[right] = s[right], s[left]
            # move the two pointers
            left, right = left+1, right-1

# test the solution
s = Solution()
test1 = ["h","e","l","l","o"]
s.reverseString(test1)
print(test1)


['o', 'l', 'l', 'e', 'h']


### 541. Reverse String II
每次移动pointer的时候，不用`i++` `i += 2k`来移动就好了。

就是在`for`循环的时候，用`for i in range(0, n, 2 * k)`

其次进行swap的时候，用`s[i: i+k] = swap(s[i: i+k])`

In [ ]:
class Solution:
    def reverseStr(self, s: str, k: int) -> str:
        """
        1. 使用range(start, end, step)来确定需要调换的初始位置
        2. 对于字符串s = 'abc'，如果使用s[0:999] ===> 'abc'。字符串末尾如果超过最大长度，则会返回至字符串最后一个值，这个特性可以避免一些边界条件的处理。
        3. 用切片整体替换，而不是一个个替换.
        """
        def swap(text: list) -> list:
            n = len(text)
            left, right = 0, n-1
            while left < right:
                text[left], text[right] = text[right], text[left]
                left, right = left+1, right-1
            return text
        
        s = list(s)
        n = len(s)

        for i in range(0, n, 2*k):
            s[i:i+k] = swap(s[i:i+k])

        return "".join(s)
    
# test the solution
s = Solution()
test = "abcdefg"
print(s.reverseStr(test, 2))

         

bacdfeg


### 151. Reverse a word in String
这题不仅仅翻转word，还要删除空格

第一步很简单，就是完全翻转所有的单词，然后再每个单词进行翻转。总共翻转两次
- he_wo -> ow_eh -> wo_he

关键怎么移除空格。要做到空间complexity `O(1)`,不要用library。用双指针来做time complexity`O(n)`。
- `fast`:
  - traverse the string, `if s[fast] != ''`
- `slow`
  - 当slow指向的不是第一个单词，`if slow != 0: s[slow] = ''; s++;`
- fast 赋值给 slow `s[slow] = s[fast]`
  - `fast++; slow++`

In [1]:
class Solution:
    def reverseWords(self, s: str) -> str:
        def swap(text: list, left: int, right: int) -> None:
            while left < right:
                text[left], text[right] = text[right], text[left]
                left, right = left+1, right-1

        # Convert string to a list
        s = list(s)
        n = len(s)

        # Reverse the entire string
        swap(s, 0, n - 1)

        # Reverse individual word
        i = 0
        while i < n:
            if s[i] != ' ':
                j = i
                while j < n and s[j] != ' ':
                    j += 1
                swap(s, i, j - 1)
                i = j  # move the pointer to next word
            i += 1

        # rmove the space
        fast, slow = 0, 0
        while fast < len(s):
            if s[fast] != ' ':
                if slow != 0:  # Ensure spaces between words
                    s[slow] = ' '
                    slow += 1
                # Copy non-space characters
                while fast < len(s) and s[fast] != ' ':
                    s[slow] = s[fast]
                    slow += 1
                    fast += 1
            else:
                fast += 1  # Skip extra spaces

        return "".join(s[:slow])  # Return only the valid part of the list as a string
    
# test the solution
s = Solution()
test = "Let's take LeetCode contest"
print(s.reverseWords(test))


contest LeetCode take Let's


## **KMP 算法**

$O(m+n)$

### 原理

解决的字符串匹配的问题。

- **String**: aabaabaaf
- **Pattern**: aabaaf


暴力解法里面，就是两个`for`循环，一个个的traverse，直到pattern string 能 match original string. 时间复杂度 $O(m*n)$

KMP 算法里面，不会从头再去匹配，而是跳到前面比如在这个案例里面，从f跳到b继续去匹配。这里涉及到一个很重要的点就是**前缀表 (Next Array)**。

1. **前缀（Prefix）**：以第一个字符开头的不包含最后一个字符的所有子串
  <pre>
  a
  aa
  aab
  aaba
  aabaa
  </pre>
2. **后缀（Suffix）**：以最后一个字符结尾的不包含第一个字符的所有子串
  <pre>
  f
  af
  aaf
  baaf
  abaaf
  </pre>
3. **部分匹配值**：模式串的前缀子串中，**最长相等**的 *前缀* 和 *后缀* 的长度
  <pre>
  a         0   单个字符无前后缀
  aa        1   aa 的最长相等前后缀是 a，长度为 1
  aab       0   aab 无相等前后缀
  aaba      1   aaba 的最长相等前后缀是 a，长度为 1
  aabaa     2   aabaa 的最长相等前后缀是 aa，长度为 2
  aabaaf    0   aabaaf 无相等前后缀
  </pre>

前缀表：
| a  | a  | b  | a  | a  | f  |
|:------:|:------:|:------:|:------:|:------:|:------:|
| 0 | 1 | 0 | 1 | 2 | 0 |

当其和原string不匹配的时候是 f`Next[5]=0`.前一个匹配的是a`Next[4]=2`,也是就是说

这个时候其实有双指针，一个指向`Text[]` 用`i`. 一个指向`Next[]`用`j`。 当Next发现有不匹配的时候，它前一个匹配的a,这时候它的Next数组下面是2，说明前面已经有两element是相同的了，所以这时候`j`应该指向了Next里面的index为2的地方，就是`Pattern[2] = b`. 与此同时，`i`还在`Text[5]=b`的位置，这时候b == b, 继续循环匹配到最后。

在实现代码的过程中，可能整体把Next数组向**右移**(这样直接按照Next不匹配的那个位置直接返回就行不用向前看一位)或者将数组每个数值都`-1`。

### 代码实现
Next：
| Pattern | a  | a  | b  | a  | a  | f  |
|:------:|:------:|:------:|:------:|:------:|:------:|:------:|
| Next | 0 | 1 | 0 | 1 | 2 | 0 |
| Index | 0 | 1 | 2 | 3 | 4 | 5 |

String：
| String | a  | a  | b  | a  | a  | b  | a  | a  | f  |
|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|
| Index`i` | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 |

这里面涉及到双指针，`i`指向后缀的首位(所以它从1开始traverse)， `j`指向前缀的末位(同时包含了j之前的最长的相同数组的长度)。 

`getNext(self, next, s)`
1. 初始化
   ```python
    next[0] = 0
    j = 0  # pointer to the prefix(next)
    for i in range(1, len(s)):  # i starts from 1
   ```
2. 前后缀不相同(遇见冲突看前一位). 因为是连续回退，用`while`
   ```python
    while j > 0 and s[i] != s[j]
        j = next[j - 1]  # look at the previous one
   ```
3. 前后缀相同
   ```python
    s[i] == s[j]
    j += 1  # s[0:j] and s[i-j+1 : i] still match
   ```
4. next数组的值更新 
    ```python
    next[i] = j
    ```

### 459. Repeated Substring Pattern

**暴力解法**

第一层`for`循环traverse那个pattern. 第二层`for`循环traverse string来对比是否存在substring pattern.

**移动匹配**

如果存在重复string， 那s + s 里面**除去首尾**，一定存在一个substring **s**
1. ss = s + s
2. ss.erase 首位，末位
3. ss.find(s)

**KMP算法**

| | a | b | a | b | a | b | a | b | 
|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|
| s | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
| 前t | 1 | 2 | 3 | 4 | 5 | 6 | - | - |
| 后f | - | - | 1 | 2 | 3 | 4 | 5 | 6 |

最长相等前后缀： $ab$:

$t = f: t01 = f01 \rightarrow s01 = s23$

$f0 = t2, f1 = t3, f01 = t23$

$t2 = f2 t3 = f3, t23 = f23 \rightarrow s23 = s45$

由推导得出，如果$字符长度 - 最长相等前后缀$ 能被 $字符长度$ 整除，说明存在这样的substring：

`len(s) % (len(s) - next[-1])`

nxt[-1]（即 nxt[len(s) - 1]）是 模式串 s 的最后一个字符的 Next 数组值。


In [3]:
from typing import List
class Solution:
    def repeatSubstringPattern(self, s: str) -> bool:
        if len(s) == 0:
            return False
        next = [0] * len(s)
        self.getNext(s, next)
        if next[-1] != 0 and len(s) % (len(s) - next[-1]) == 0:
            return True
        return False
    
    def getNext(self, s: str, next: List[int]) -> List[int]:
        next[0] = 0
        j = 0
        for i in range(1, len(s)):
            while j > 0 and s[i] != s[j]:
                j = next[j - 1]
            if s[i] == s[j]:
                j += 1
            next[i] = j
        return next
    
# test the solution
s = Solution()
test = "abab"
print(s.repeatSubstringPattern(test))
s2 = Solution()
test2 = "aba"
print(s2.repeatSubstringPattern(test2))

True
False
